# A notebook with evaluations of the validation cases.
The evaluation of the cases of the unit disc, the unit square and static isospectral domains is implemented here.
## The Unit Disc
The numeric results were computed with COMSOL. The evaulation is as follows:
- Plot the evloution of the eigenvalues $\lambda_0, \lambda_2$.
- Eigenvalues distinguished by *color*
- Plot the growth factors for $\frac{R_0\omega}{c},2\cdot\left(\frac{R_0\omega}{c}\right)$ with $R_0=1$.
- x-axis $\frac{R_0\omega}{c}$ dimensionless.
- y-axis $\vert\lambda_{num} - \lambda_{an,stat}\vert\;\left[\frac{1}{m}\right]$.

## The Unit Square

## Static Isospectral Domains
Results obtained with COMSOL.
- Plot the deviation of the eigenvalues of domain 1 and domain 2 from Driscoll as a bar chart.
- x-axis : Eigenmode index
- y-axis: $\vert\lambda_{num}-\lambda_{ref}\vert$.

## Sunada and Harayama Cavity
Results obtained with COMSOL.
- Plot the deviation of the eigenvalues, scaled by the mean radius of the cavity $R_0$, as a function of the non-dimensional angular frequency $\frac{R_0\omega}{c}$.
- Add a slope triangle with slope $1$ to indicate the growth.
- Perhaps use interpolation of the two parts of the graph to determine the (non-dimensional) angular frequency where the cavity becomes a "circle".

## Polynomial Degree Studies
Using results obtained with own code draw the behaviour of the anisospectrality of the ground state $\lambda_1$ and the ninth excited state $\lambda_9$ for the first pair of isospectral domains for different orders of ansatz polynomials. For vacuum and diamond separately.

## Basic Studies
Manually run tests

Load the necessary libraries

In [1]:
import numpy as np
import scipy as sp
import os, shutil, sys, re
from matplotlib import pyplot as plt
from scipy.constants import codata
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
from slope_marker import *
from PlotFunctions import readEigenvalueData
from scipy import special

Store the current directory as the point of origin

In [2]:
origin = os.getcwd();

Fetch the CODATA value for the speed of light

In [3]:
c = sp.constants.value('speed of light in vacuum');

## Further notes
The plots 2,3,4,5 are to be made for vacuum and diamond (data subdirectories Exp_0 and Exp_2).
Whenever 2 colors are required (to distinguish domains) the following are used:  darkred, darkgreen.

Similarly, the two distinct line styles are: solid, dotted

Similarly, the two distinct markers are: 'o' - filled circle, 'x' - cross

The resolution for the images should be 1000 dpi.

The indices of the ground state and the 9th excited state are -1, 1 by merit of ARPACK
returning a list of eigenvalues sorted from smaller to larger considering the signs.

*It is assumed that the data in the directories has been pre-evaluated using EvalScript.py* Such that the collective data is stored in the .npz files.

** The side length for the first pair is 2, not 1 **

The $\omega$ values used in eigenmode plots are:
$\omega_1 = 661.539$
$\omega_2 = 58891200$
$\omega_3 = 132919000$

In [4]:
vacuumSubdir = "Exp_0"
diamondSubdir = "Exp_1"
color1 = 'darkred'
color2 = 'darkgreen'
ls1 = 'solid'
ls2 = ':'
mark1 = 'o'
mark2 = 'd'
lsize=14
resolution = 1000
GSIdx = -1
NSIdx = 1
lw = 2.5
omega1 = 661.539
omega2 = 58891200
omega3 = 132919000

Define evaluation functions

In [5]:
def ReadInput(datafiles, readOmega = False, omegaIdx=0, evIdx=-1):
    EvInput = []
    XValInput = []
    for k in range(len(datafiles)):
        Input = open(datafiles[k], 'r')
        RawDataEv = []
        RawXVal = []
        for line in Input:
            data = line.split();
            if data[0]=="%":
                continue;
            else:
                RawDataEv.append(float(data[evIdx]));
                if readOmega==True:
                    RawXVal.append(float(data[omegaIdx]));

        EvInput.append(RawDataEv)
        XValInput.append(RawXVal)
        Input.close()
    print np.shape(EvInput)
    if readOmega == True:
        return XValInput, EvInput;
    else:
        return EvInput;
    
def EVRectangle(l=1, m=1, Lx=1, Ly=1, Neumann=False):
    if Neumann==True:
        return np.pi*np.sqrt( (l/Lx)**2+(m/Ly)**2 )
    else:
        return np.pi*np.sqrt( (l+1/Lx)**2+(m+1/Ly)**2 )
        
def EVDisk(m=1, N=1, r=1, Neumann=False):
    if Neumann==True:
        return special.jnp_zeros(m, N)/r;
    else:
        return special.jn_zeros(m, N)/r;
    
def AnalyticEVRectangle(Neig=10, Neumann=False, Lx=1, Ly=1 ):
    k = 0
    EVlist = []
    maxIter = int( np.ceil( 0.5*(1+np.sqrt(1.0+8*Neig))))
    for i in xrange(maxIter):
        if k>=Neig:
            break
        for j in xrange(i+1):
            if k>= Neig:
                break;
            if j==i:
                EVlist.append(EVRectangle(i,j, Lx,Ly, Neumann))
                EVlist.append(EVRectangle(i,j, Lx,Ly, Neumann))
                k+=2;
                continue;
            else:
                EVlist.append(EVRectangle(i,j, Lx,Ly, Neumann))
                EVlist.append(EVRectangle(j,i, Lx,Ly, Neumann))
                EVlist.append(EVRectangle(i,j, Lx,Ly, Neumann))
                EVlist.append(EVRectangle(j,i, Lx,Ly, Neumann))
                k+=4

    return np.array(EVlist)
            
def AnalyticEVCircle(Neig=10, Neumann=False, R=1):
    EVList = []
    maxM = Neig/2
    #collect eigenvalues
    for i in xrange(maxM):
        EVList+=EVDisk(i, maxM, R, Neumann).tolist()
        if i != 0:
            EVList+=EVDisk(i, maxM, R, Neumann).tolist()
    #sort the list
    if Neumann==True:
        EVList.append(0.0)
    EVList.sort() 

    return np.array(EVList[0:Neig])

## The Unit Disc

Fetch the data 

In [6]:
CircleDir = "../data/COMSOL/Circle"
os.chdir(CircleDir);
datafiles = ["EV_Stat_D.txt", "EV_Stat_N.txt"];
Eval = ReadInput(datafiles);

(2, 20)


Split the eigenvalues and retain only those obtained for the Dirichlet boundary condition

In [7]:
EvalArr = np.array(Eval);
EigVal = EvalArr[0,:];

Determine analytic eigenvalues

In [8]:
EVAnD = AnalyticEVCircle(len(EigVal));

Sort

In [9]:
#duplicate
tmp = np.array([EVAnD, EVAnD])
EVAnD = tmp.flatten()
#sort
EVAnD.sort()
EigVal.sort()

#compute the absolute deviation
Diff = abs(EigVal[0:len(EigVal)] - EVAnD[0:len(EigVal)])

Read the results of the simulation and return to origin

In [10]:
datafiles = ["Lin_GS_D.txt", "Lin_GS_N.txt",
    "Lin_MS_D.txt", "Lin_MS_N.txt"];
omega, Eval = ReadInput(datafiles, True);

#choose the ground and maximum states
GSEVD_num = np.array(Eval[0])
MSEVD_num = np.array(Eval[2])
os.chdir(origin);

(4, 20)


Determine the analytic counterparts to the eigenvalues and the appropriate masks

In [11]:
#determine the analytic counterparts to the eigenvalues
mask = np.where(abs(EVAnD - GSEVD_num[0])<1e-4)
GSEVD_an = EVAnD[mask[0][0]]
    
mask = np.where(abs(EVAnD - MSEVD_num[0])<1e-4)
MSEVD_an = EVAnD[mask[0][0]]
#compute the deviation of the Dirichlet eigenvalues
GSdevD = abs( GSEVD_num - GSEVD_an)
MSdevD = abs( MSEVD_num - MSEVD_an)
    
#mask for omega
maskGS = np.where(abs(GSEVD_num - GSEVD_an)<1e-1)
maskMS = np.where(abs(MSEVD_num - MSEVD_an)<1e-1)
#store for later plots
maskGSD = maskGS
maskMSD = maskMS

Parameters for plots

In [12]:
R0 = 1.0
#linear increase plots
x = np.array(omega[0])*R0/c;
yLin1 = x;
yLin2 = 2*x;

Plot

In [13]:
fig, ax = plt.subplots()
#numerical results
ax.loglog(np.array(omega[0])[maskGS[0]]*R0/c, GSdevD, marker=mark1, color=color1, linestyle=ls1, lw=lw);
ax.loglog(np.array(omega[2])[maskMS[0]]*R0/c, MSdevD, marker=mark2, color=color2, linestyle=ls2, lw=lw);

#analytic results
ax.loglog(x,yLin1, 'k--', x, yLin2, 'k-');

plt.xlabel("$\\frac{R_0\omega}{c}$", size='xx-large')
plt.ylabel("$|\lambda_{num}-\lambda_{an,stat}|\;\left[\\frac{1}{m}\\right]$", size='x-large')
plt.grid()

plt.legend(("Ground state", "Highest computed", "$\\frac{R_0\omega}{c}$", "$2\cdot\left(\\frac{R_0\Omega}{c}\\right)$"),
          loc='best');
plt.title("Deviation from static analytic eigenvalue, Dirichlet BC");
#fit
plt.xlim(min(x), max(x));
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
plt.tight_layout();
plt.savefig("Results_UnitDisc.eps", format='eps', dpi=resolution)
plt.show()

Print the maximal deviation from the predicted analytic evolution of the second state

In [14]:
EVDev = abs( np.array(omega[0])[maskGS[0]]*R0/c - yLin2)
max(EVDev)

0.033356409519815208

## The Unit Square

Read the data

In [15]:
SquareDir = "../data/Rectangle/Exp_0"
os.chdir(SquareDir);

omegaNodal = [];
EVNodal = [];
inputfile = open("output.dat", 'r')
lineIdx=0;
for line in inputfile:
    data = line.split()
    if lineIdx==0 :
        for el in data:
            omegaNodal.append(float(el));
    else:
        EigValRow = []
        for el in data:
            EigValRow.append(float(el.split('+')[0]));
        EVNodal.append(EigValRow);
    lineIdx+=1;

EVNodalArray = abs(np.array(EVNodal));
omegaArrNodal = np.array(omegaNodal);
os.chdir(origin)

Read the static data and compute the deviations

Load the modal decomposition

In [17]:
ModalDir="../data/Modal"
os.chdir(ModalDir)
omega = []
GSlist = []
FirstStateList = []
SecondStateList = []
#read the ground state
inputfile = open("GroundStateDeviationMathematica.dat", 'r');
for line in inputfile:
    data = line.split();
    omega.append(float(data[0]));
    GSlist.append(float(data[-1]));
inputfile.close
#read the first excited state
inputfile = open("FirstStateDeviationMathematica.dat", 'r');
for line in inputfile:
    data = line.split()
    FirstStateList.append(float(data[-1]));
inputfile.close()
#read the second excited state
inputfile = open("SecondStateDeviationMathematica.dat", 'r');
for line in inputfile:
    data = line.split()
    SecondStateList.append(float(data[-1]));
inputfile.close()

os.chdir(origin)

Convert to arrays

In [18]:
omegaModal = np.array(omega);
GSmodal = abs(np.array(GSlist));
FSmodal = abs(np.array(FirstStateList));
SSmodal = abs(np.array(SecondStateList));

Plot

In [19]:
plt.loglog(omegaModal, GSmodal, 'r-+', lw=lw)

plt.loglog(omegaModal, SSmodal, 'b-+', lw=lw)
plt.grid()
plt.legend(("$\lambda_1$", "$\lambda_3$"), loc='best')
plt.xlabel("$\\frac{R_0\omega}{c}$", size='x-large')
plt.ylabel("$\\frac{\Delta\lambda(\omega)}{\lambda(0)}$", size='large')
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
plt.tight_layout();
plt.savefig("Modal_Deviation.eps", format='eps', dpi=resolution)
plt.show()

In [20]:
R0 = np.sqrt(2.0)
xVal = omegaArrNodal[1:]*R0/c;

plt.loglog(xVal, abs(EVNodalArray[-1,1:]-2*np.pi**2)/(2*np.pi**2), 'r-+', lw=lw)

plt.loglog(xVal, abs(EVNodalArray[-7,1:]-8*np.pi**2)/(8*np.pi**2), 'b-+', lw=lw)
plt.grid()
plt.legend(("$\lambda_1$", "$\lambda_3$"), loc='best')
plt.xlabel("$\\frac{R_0\omega}{c}$", size='x-large')
plt.ylabel("$\\frac{\Delta\lambda(\omega)}{\lambda(0)}$", size='large')
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
plt.tight_layout();
plt.savefig("Nodal_Deviation.eps", format='eps', dpi=resolution)
plt.show()

In [21]:
plt.loglog(omegaModal, GSmodal, 'r-', lw=lw)
plt.loglog(omegaModal, SSmodal, 'b-', lw=lw)

plt.loglog(xVal, abs(EVNodalArray[-1,1:]-2*np.pi**2)/(2*np.pi**2), 'r--', lw=lw)
plt.loglog(xVal, abs(EVNodalArray[-7,1:]-8*np.pi**2)/(8*np.pi**2), 'b--', lw=lw)

plt.grid()
plt.legend(("$\lambda_1$", "$\lambda_3$"), loc='best')
plt.xlabel("$\\frac{R_0\omega}{c}$", size='x-large')
plt.ylabel("$\\frac{\Delta\lambda(\omega)}{\lambda(0)}$", size='large')
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
plt.tight_layout();
plt.savefig("Nodal_Nodal_CompositePlot.eps", format='eps', dpi=resolution)
plt.show()

## Static Isospectral Domains


The reference values

In [25]:
#define the static eigenvalues of the ID's according to Driscoll
StatEVReference = np.array([2.53794399980, 9.20929499840, 14.3138624643,
20.8823950433, 24.6740110027,3.65550971352, 10.5969856913, 15.8713026200,
21.2480051774, 26.0802400997,5.17555935622, 11.5413953956, 16.9417516880,
22.2328517930, 27.3040189211,6.53755744376, 12.3370055014, 17.6651184368,
23.7112974848, 28.1751285815,7.24807786256, 13.0536540557, 18.9810673877,
24.4792340693, 29.5697729132])
StatEVReference.sort();

Redefine the read function

In [26]:
def ReadInput(datafiles):
    EvInput = []
    xValues = []

    for k in range(len(datafiles)):
        Input = open(datafiles[k], 'r')
        RawDataEv = []
        RawDataXval = []
        for line in Input:
            data = line.split();
            if data[0]=="%":
                continue;
            else:
                RawDataEv.append(float(data[-1]));
                RawDataXval.append(float(data[0]));
        xValues.append(RawDataXval)
        EvInput.append(RawDataEv)

    return [xValues, EvInput];

Go to the directory where the solutions are stored

In [27]:
StatIDDir = "../data/COMSOL/ID1"
datafiles = ["ID1_1_Vacuo_D_EV_Stat.txt", "ID1_2_Vacuo_D_EV_Stat.txt"]

Read the data

In [28]:
os.chdir(StatIDDir);
xValues, EvInput = ReadInput(datafiles);
EvArray=np.array(EvInput);
xValArray=np.array(xValues);
os.chdir(origin);

Sort, square and pick only real eigenvalues

In [29]:
EvArray.sort();
UniqueValues = np.shape(EvArray)[1]/2;
#square
ID1EV = EvArray[0,0:UniqueValues]**2;
ID2EV = EvArray[1,0:UniqueValues]**2;
#pick only real EV
mask1 = np.where(ID1EV>1)[0];
mask2 = np.where(ID2EV>1)[0];
ID1EV = ID1EV[mask1];
ID2EV = ID2EV[mask2];
#sort in ascending order
ID1EV.sort();
ID2EV.sort();

Compare and compute absolute deviation

In [30]:
#compare data
ID1deviation = np.zeros(len(ID1EV));
ID2deviation = np.zeros(len(ID2EV));

EVdifference = abs(ID1EV - ID2EV);
#compute absolute deviation
for k in xrange( len(ID1EV) ) :
    ID1deviation[k] = abs( ID1EV[k] - StatEVReference[k] );
for k in xrange( len(ID2EV) ) :
    ID2deviation[k] = abs( ID2EV[k] - StatEVReference[k] );

Plot the absolute error and the deviation of the eigenvalues

In [31]:
#plot parameters
bar_width = 0.35
index = np.arange(len(ID1EV))

#plot
plt.subplot(1,2,1)

#draw the absolute deviation
plt.bar(index, ID1deviation, bar_width, color=color1, label="Domain 1", align='center', log=True);
plt.bar(index+bar_width, ID2deviation, bar_width, color=color2, label="Domain 2", align='center', log=True);
plt.legend(loc='upper left', framealpha=0.7);

plt.grid();
plt.xticks(index + bar_width/2, (index+1))
plt.xlabel("Eigenvalue index");
plt.ylabel("$|\lambda^2_{num} - \lambda^2_{ref}|\;\left[ \\frac{1}{m} \\right]$", size='large');

plt.subplot(1,2,2);
#draw the anisospectrality
plt.bar(index, EVdifference, bar_width,color='r', align='center', log=True);

plt.grid();
plt.xlabel("Eigenvalue index");
plt.ylabel("$|\left( \lambda^{(1)} \\right)^2 - \left( \lambda^{(2)} \\right)^2|\;\left[ \\frac{1}{m} \\right]$",\
           size='large');

plt.xticks(index, (index+1))
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
plt.tight_layout();
plt.savefig("Results_StaticIDs.eps", format='eps', dpi=resolution)
plt.show()

Results obtained using own code.

The data is organized in an array as follows

|Method | $\lambda_1$ | ... | $\lambda_{10}$ |
| :--- | :---: | :---: | :---: |
| FEM $N_{ref} = 1$ | ... | ... | ... |
| ... | ... | ... | ... |
| FEM $N_{ref} = 7$ | ... | ... | ... |

Domain 1

In [30]:
EVID1own=np.array([[-13.2295132778,-12.5686877302, -11.7628333305, -10.7590025818,-9.30075777195 , -7.30422132186, -6.55326703592, -5.20879921043, -3.67242406704, -2.55277675974 ],
[-13.0734575132, -12.3423459403 , -11.5476473828, -10.605930045 ,-9.21586760371, -7.25889536973, -6.54061574393 ,-5.18683224308,-3.66136212899, -2.54371387797],
[-13.0604930202, -12.3370949844 , -11.542307298, -10.5996960358, -9.2113129933, -7.25227618399, -6.53855768074, -5.1800174735, -3.65778035856, -2.54022155518 ],
[-13.0563198596, -12.3370069135, -11.5416972502, -10.5980231311, -9.2100754866, -7.24974513716, -6.53792753306, -5.17732468003 , -3.65640355771, -2.53884574699],
[-13.0547053567,-12.3370055234 , -11.5415084911 , -10.5973925501 , -9.20960212686  , -7.24873928906 , -6.53770019047  , -5.17625901735 , -3.65586327765,-2.5383014885],
[-13.0540701647 , -12.3370055017 , -11.5414392481  , -10.5971463816,-9.20941645129   , -7.2483402705  , -6.53761344348,-5.17583685038 , -3.65564983946 , -2.53808580606 ],
[-13.053819007,-12.3370055013 , -11.5414126355 ,-10.5970493375 ,-9.20934312674 , -7.24818198334  , -6.53757956473,-5.17566945154 , -3.65556529274 , -2.53800026511]]);

Domain 2

In [31]:
EVID2own=np.array([[-13.5050123448, -12.5671596432, -11.7526386517, -10.7666010439, -9.29880094937, -7.30835862312 , -6.56864607574, -5.21473188956, -3.67432580727 , -2.55370998204  ],
[-13.0791611795,-12.3411356147, -11.5476505776 , -10.6066157906 , -9.21632749328 , -7.2596343083 ,-6.54095033937, -5.18728485677, -3.66145057259, -2.54377547446],
[-13.0606576382, -12.3370744446,-11.5423165973 , -10.5997320719, -9.2113497924, -7.25232910108, -6.53856774037 , -5.18004933815,-3.65778765446,-2.54022762534],
[-13.056330188 , -12.3370065856 , -11.5416981459  , -10.598025924, -9.21007824679, -7.24974922844, -6.53792815338, -5.17732738577 , -3.65640429748 ,-2.53884637849 ],
[-13.0547063471 ,-12.3370055183, -11.5415085749 ,-10.5973928103 , -9.20960236242, -7.24873965014, -6.53770024637, -5.17625927169 ,-3.65586335335   , -2.53830155305],
[-13.0540702653  , -12.3370055016, -11.5414392562 , -10.5971464071,-9.2094164732, -7.2483403047,-6.53761344897 , -5.17583687518 ,-3.6556498471,-2.53808581255 ],
[-13.0538190171 , -12.3370055013,-11.5414126363,-10.59704934, -9.20934312886, -7.24818198667, -6.53757956527, -5.17566945399 , -3.65556529351, -2.53800026576]])

Absolute and relative errors of the eigenvalues computed with own software

In [32]:
#sort
EVID1own = abs(EVID1own);
EVID2own = abs(EVID2own);
EVID1own.sort();
EVID2own.sort();

In [33]:
absErrID1own = np.zeros(np.shape(EVID1own));
absErrID2own = np.zeros(np.shape(EVID2own));
for k in range(np.shape(EVID1own)[0]):
    absErrID1own[k] = abs(EVID1own[k,:] - StatEVReference[:len(EVID1own[k,:])])
for k in range(np.shape(EVID2own)[0]):
    absErrID2own[k] = abs(EVID2own[k,:] - StatEVReference[:len(EVID2own[k,:])])

In [34]:
relErrID1own = np.zeros(np.shape(EVID1own));
relErrID2own = np.zeros(np.shape(EVID2own));
for k in range(np.shape(EVID1own)[0]):
    relErrID1own[k] = absErrID1own[k]/StatEVReference[:len(EVID1own[k,:])]
for k in range(np.shape(EVID2own)[0]):
    relErrID2own[k] = absErrID2own[k]/ StatEVReference[:len(EVID2own[k,:])]

Plot the deviation from the reference values for the highest refinement possible

In [35]:
index = np.arange(len(EVID1own[k,:]));

#plot
plt.subplot(1,2,1)

#draw the absolute deviation
plt.bar(index, absErrID1own[-1], bar_width, color=color1, label="Domain 1", align='center', log=True);
plt.bar(index+bar_width, absErrID2own[-1], bar_width, color=color2, label="Domain 2", align='center', log=True);
plt.legend(loc='upper left', framealpha=0.6);

plt.grid();
plt.xticks(index + bar_width/2, (index+1))
plt.xlabel("Eigenvalue index");
plt.ylabel("$|\lambda^2_{num} - \lambda^2_{ref}|\;\left[ \\frac{1}{m} \\right]$", size='large');

plt.subplot(1,2,2);
#draw the anisospectrality
plt.bar(index, abs(EVID1own[-1,:] - EVID2own[-1,:]), bar_width,color='r', align='center', log=True);

plt.grid();
plt.xlabel("Eigenvalue index");
plt.ylabel("$|\left( \lambda^{(1)} \\right)^2 - \left( \lambda^{(2)} \\right)^2|\;\left[ \\frac{1}{m} \\right]$",\
           size='large');

plt.xticks(index, (index+1))
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
plt.tight_layout();
plt.savefig("Results_Own_Nref7_StaticIDs.eps", format='eps', dpi=resolution)
plt.show()

Do the same for a mediocre refinement

In [36]:
index = np.arange(len(EVID1own[k,:]));
refIdx = 3
#plot
plt.subplot(1,2,1)

#draw the absolute deviation
plt.bar(index, absErrID1own[refIdx], bar_width, color=color1, label="Domain 1", align='center', log=True);
plt.bar(index+bar_width, absErrID2own[refIdx], bar_width, color=color2, label="Domain 2", align='center', log=True);
plt.legend(loc='upper left', framealpha=0.6);

plt.grid();
plt.xticks(index + bar_width/2, (index+1))
plt.xlabel("Eigenvalue index");
plt.ylabel("$|\lambda^2_{num} - \lambda^2_{ref}|\;\left[ \\frac{1}{m} \\right]$", size='large');

plt.subplot(1,2,2);
#draw the anisospectrality
plt.bar(index, abs(EVID1own[refIdx,:] - EVID2own[refIdx,:]), bar_width,color='r', align='center', log=True);

plt.grid();
plt.xlabel("Eigenvalue index");
plt.ylabel("$|\left( \lambda^{(1)} \\right)^2 - \left( \lambda^{(2)} \\right)^2|\;\left[ \\frac{1}{m} \\right]$",\
           size='large');

plt.xticks(index, (index+1))
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
plt.tight_layout();
plt.savefig("Results_Own_Nref%d_StaticIDs.eps"%(refIdx), format='eps', dpi=resolution)
plt.show()

## Sunada and Harayama Cavity

Fetch the data. The values of $\omega$ from the second file will be ignored since they are identical to those from the first file.
When the loading is done the script changes back to the directory of origin.

In [37]:
SHDir = "../data/COMSOL/SH"
os.chdir(SHDir);
inputfile = open("Eigenfreq_1__1RotDisc.txt", 'r');
omega1 = []
EV1 = []
for line in inputfile:
    data = line.split();
    if data[0] == "%":
        continue;
    else:
        omega1.append( float(data[0]));
        EV1.append( float(data[-1]));

inputfile.close()

#second
inputfile = open("Eigenfreq_2_1_RotDisc.txt", 'r');
omega2 = []
EV2 = []
for line in inputfile:
    data = line.split();
    if data[0] == "%":
        continue;
    else:
        omega2.append( float(data[0]));
        EV2.append( float(data[-1]));

inputfile.close()

os.chdir(origin);

Cast into arrays and compute the difference

In [38]:
omegaArray = np.array(omega1)
EV1Array = np.array(EV1)
EV2Array = np.array(EV2)
EVDiff = abs(EV1Array - EV2Array)

Determine the permutation of the indices necessary to put the angular frequencies into an ascending order,  then re-order angular frequencies and eigenvalues using the permutation obtained.

In [39]:
omegaIndices = omegaArray.argsort()
EVDiffSorted = EVDiff[omegaIndices[::1]]
omegaSorted = omegaArray[omegaIndices[::1]];

Plot the eigenvalues

The eigenvalues carry the dimension of a wave-vector, $\left[\frac{1}{L}\right]$. Being in vacuum we can use the relation $\omega = c\Vert \mathbf{k}\Vert_2$ to obtain the eigenfrequencies. 
Thus $\frac{R_0\vert \Delta\omega\vert}{c} = R_0\vert \Delta \lambda\vert$.

In [40]:
R0 = 6.2866*1e-6;
xVal = R0*omegaSorted/c;
yVal = R0*EVDiffSorted;


fig,ax = plt.subplots()
ax.loglog(xVal, yVal, linestyle=ls1, lw=lw, marker='o', color=color1);
ax.set_xlabel("$\\frac{R_0\omega}{c}$", size='x-large')
ax.set_ylabel("$\\frac{R_0| \Delta\omega|}{c}$", size='large')

ax.set_xlim(min(xVal), max(xVal));
ax.set_ylim(5e-6,1e-3);

ax.grid()

#add slope marker
slope_marker((2e-6,2e-4), (1,1), 0.05, 0.15, ax, invert=True, ec='black', linestyle='solid', lw='1.5', fill=False)
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
plt.tight_layout();

plt.savefig("SH_EigenmodeDeviation_AdaptedToSH.eps", format='eps', dpi=resolution)
plt.show()

-------------------------
## Polynomial Degree Studies

The directories where the data is stored in subdirectories. Using data obtained with the full PDE.

The plots will require the radii of the circumcircles of the domains, too.

In [41]:
R0Dict = {"ID1_1":3.6651, "ID1_2":3.4667, "ID2_1":4.55, "ID2_2":4.8365 };

In [42]:
P1Dir="../data/ID1/P1_Full";
P2Dir="../data/ID1/P2_Full";
P3Dir="../data/ID1/P3_Full";

#subdirectories for vacuum and diamond data
vacuumSubdir="/Exp_0";
diamondSubdir="/Exp_2";

Fetch data for vacuum and return to the origin directory.

In [43]:
os.chdir(P1Dir+vacuumSubdir);
P1data = np.load("Results_Exp_0.npz");
os.chdir(P2Dir+vacuumSubdir);
P2data = np.load("Results_Exp_0.npz");
os.chdir(P3Dir+vacuumSubdir);
P3data = np.load("Results_Exp_0.npz");
os.chdir(origin);

Load the angular frequencies, the anisospectrality values as well as the refinement level for the highest level of refinement used.

In [44]:
#P1 FE
P1Aniso = P1data['Data'][-1,];
P1MaxRef = P1data['N'][-1];
omega = P1data['omega'];
#P2 FE
P2Aniso = P2data['Data'][-1,];
P2MaxRef = P2data['N'][-1];
#P3 FE
P3Aniso = P3data['Data'][-1,];
P3MaxRef = P3data['N'][-1];

Plot

In [45]:
R0 = max( R0Dict["ID1_1"], R0Dict["ID1_2"]);
x = R0*omega/c;

fig, ax = plt.subplots();
#ground state
ax.loglog(x[1:], P1Aniso[GSIdx,1:], color=color1, lw=lw, linestyle=ls1, marker=mark1);
ax.loglog(x[1:], P2Aniso[GSIdx,1:], color=color2, lw=lw, linestyle=ls1, marker=mark1);
ax.loglog(x[1:], P3Aniso[GSIdx,1:], color='darkblue', lw=lw, linestyle=ls1, marker=mark1);
#ninth excited
ax.loglog(x[1:], P1Aniso[NSIdx,1:], color=color1, lw=lw, linestyle=ls2, marker=mark2);
ax.loglog(x[1:], P2Aniso[NSIdx,1:], color=color2, lw=lw, linestyle=ls2, marker=mark2);
ax.loglog(x[1:], P3Aniso[NSIdx,1:], color='darkblue', lw=lw, linestyle=ls2, marker=mark2);

ax.grid();
ax.set_xlabel("$\\frac{R_0\omega}{c}$", size='x-large');
ax.set_ylabel("$|(\lambda^{(1)})^2 - (\lambda^{(2)})^2|\;\left[\\frac{1}{m}\\right]$", size='x-large')

#legend
ax.legend(("$\lambda_0\;\mathcal{P}_1$","$\lambda_0\;\mathcal{P}_2$","$\lambda_0\;\mathcal{P}_3$",\
          "$\lambda_9\;\mathcal{P}_1$","$\lambda_9\;\mathcal{P}_2$","$\lambda_9\;\mathcal{P}_3$"),\
         loc='lower right', framealpha=0.7)
#fit
ax.set_xlim(min(x), max(x));

#shade the unphysical region
ax.axvspan(5*1e-1, max(x), facecolor='none', hatch='x', alpha=0.2)
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
fig.tight_layout()
#save and draw
fig.savefig("ID1_FullEquation_PolydegStudy_Vacuum.eps", format='eps', dpi=resolution);
plt.show()

----------------------
Now the same for diamond, this time load and extract data in one go.

In [46]:
os.chdir(P1Dir+diamondSubdir);
P1data = np.load("Results_Exp_2.npz");
os.chdir(P2Dir+diamondSubdir);
P2data = np.load("Results_Exp_2.npz");
os.chdir(P3Dir+diamondSubdir);
P3data = np.load("Results_Exp_2.npz");
os.chdir(origin);

#P1 FE
P1Aniso = P1data['Data'][-1,];
P1MaxRef = P1data['N'][-1];
omega = P1data['omega'];
#P2 FE
P2Aniso = P2data['Data'][-1,];
P2MaxRef = P2data['N'][-1];
#P3 FE
P3Aniso = P3data['Data'][-1,];
P3MaxRef = P3data['N'][-1];

Plot

In [47]:
R0 = max( R0Dict["ID1_1"], R0Dict["ID1_2"]);
x = R0*omega/c;

fig, ax = plt.subplots();
#ground state
ax.loglog(x[1:], P1Aniso[GSIdx,1:], color=color1, lw=lw, linestyle=ls1, marker=mark1);
ax.loglog(x[1:], P2Aniso[GSIdx,1:], color=color2, lw=lw, linestyle=ls1, marker=mark1);
ax.loglog(x[1:], P3Aniso[GSIdx,1:], color='darkblue', lw=lw, linestyle=ls1, marker=mark1);
#ninth excited
ax.loglog(x[1:], P1Aniso[NSIdx,1:], color=color1, lw=lw, linestyle=ls2, marker=mark2);
ax.loglog(x[1:], P2Aniso[NSIdx,1:], color=color2, lw=lw, linestyle=ls2, marker=mark2);
ax.loglog(x[1:], P3Aniso[NSIdx,1:], color='darkblue', lw=lw, linestyle=ls2, marker=mark2);

ax.grid();
ax.set_xlabel("$\\frac{R_0\omega}{c}$", size='x-large');
ax.set_ylabel("$|(\lambda^{(1)})^2 - (\lambda^{(2)})^2|\;\left[\\frac{1}{m}\\right]$", size='x-large')

#legend
ax.legend(("$\lambda_0\;\mathcal{P}_1$","$\lambda_0\;\mathcal{P}_2$","$\lambda_0\;\mathcal{P}_3$",\
          "$\lambda_9\;\mathcal{P}_1$","$\lambda_9\;\mathcal{P}_2$","$\lambda_9\;\mathcal{P}_3$"),\
         loc='lower right', framealpha=0.7)
#fit
ax.set_xlim(min(x), max(x));

#shade the unphysical region
ax.axvspan(5*1e-1, max(x), facecolor='none', hatch='x', alpha=0.2)
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
fig.tight_layout()
#save and draw
fig.savefig("ID1_FullEquation_PolydegStudy_Diamond.eps", format='eps', dpi=resolution);
plt.show()

--------------------
## Basic Studies

First a manual collection of the errors for simple PDE, Simple PDE with trigonometric functions and a PDE with transport.

The order is $\mathcal{P}_1,\mathcal{P}_2,\mathcal{P}_3$

In [48]:
#mesh width
hSimplePoly = np.array([0.25,0.125,0.0625,0.03125,0.015625]);
#L2 error
L2Error_SimplePolyQC = np.array(
    [[0.00304888,0.000844363,0.000216787,5.45618e-05,1.36634e-05],
     [0.000259694,3.23212e-05,4.02961e-06,5.03446e-07,6.29265e-08],
     [2.2221e-05,1.36095e-06,8.27762e-08,5.07671e-09,3.13829e-10]])
#H1 error
H1Error_SimplePolyQC = np.array([[0.0361373, 0.0190755,0.00967272,0.00485357,0.00242895],
                                [0.00758449,0.0019903,0.000504271,0.000126517,3.16582e-05],
                                [0.00091226,0.000115685,1.44012e-05,1.7908e-06,2.23076e-07]])
#L-inf error
#LInfError_SimplePolyQC = np.array( [[0.00186826,0.000501065, 0.000129457,3.2867e-05, 8.2321e-06],
#                                   []])
f  = np.array([0.482717, 0.482717,0.482717])
#create the custom ticks for the x-axis.
NrefSimple = np.array( [int(np.log2(x)) for x in 1/hSimplePoly] );
xTicksSimple = [1.0/(2**k) for k in NrefSimple]
xTickLabSimple =["$\\frac{1}{2^{%d}}$"%(k) for k in NrefSimple]

Next for the poisson equation with trigonometric solutions.

In [49]:
hTrig=np.array([0.5,0.25,0.125,0.0625,0.03125])
#L2
L2Error_TrigQC = np.array([[0.500005,0.16208,0.0462109,0.0119672,0.00301894],
                           [0.116052,0.0159398,0.00206637,0.00026105,3.2727e-05],
                           [0.0285441,0.00192692,0.000116973,7.14083e-06,4.41458e-07]])
#H1
H1Error_TrigQC = np.array([[3.5478,1.89355,1.00272,0.508897,0.25541],
        [1.41288,0.450094,0.119876,0.0305061,0.00766213],
        [0.523477,0.0735556,0.00939114,0.00117461,0.000146621]])
#RHS L2 norm
f= np.array([24.674,24.674,24.674])
#
NrefTrig = np.array( [int(np.log2(x)) for x in 1/hTrig] );
xTicksTrig = [1.0/(2**k) for k in NrefTrig]
xTickLabTrig =["$\\frac{1}{2^{%d}}$"%(k) for k in NrefTrig]

Finally the poisson equation with a gyroscopic term and a polynomial solution

In [50]:
hTrans = np.array([0.25,0.125,0.0625,0.03125,0.015625])
#L2
L2Error_Trans = np.array([[0.00246472,0.000670436,0.000171558,4.31474e-05,1.08032e-05,],
                           [0.0323943,3.19154e-05,4.01628e-06,5.03027e-07,6.29134e-08],
                           [2.21742e-05,1.36029e-06,8.27662e-08,5.07655e-09,3.13826e-10]])
#H1
H1Error_Trans = np.array([[0.0367046,0.0191482,0.00968192,0.00485472,0.00242909],
        [0.937899,0.00199385,0.00050457,0.000126538,3.16596e-05],
        [0.000916595,0.000115841,1.44061e-05,1.79094e-06,2.23081e-07]])
#L-inf norm 0.00154509,0.000368246,9.18926e-05,2.28703e-05,5.70762e-06
#RHS L2 norm
f= np.array([0.456846,0.456846,0.456846])
#create the custom ticks for the x-axis.
NrefTrans = np.array( [int(np.log2(x)) for x in 1/hTrans] );
xTicksTrans = [1.0/(2**k) for k in NrefTrans]
xTickLabTrans =["$\\frac{1}{2^{%d}}$"%(k) for k in NrefTrans]

Now plot the $L^2$ errors

In [54]:
fig, ax = plt.subplots()

ax.loglog(hSimplePoly[::-1],L2Error_SimplePolyQC[0,::-1],  lw=lw, color=color1, linestyle=ls1, marker=mark1);
ax.loglog(hSimplePoly[::-1],L2Error_SimplePolyQC[1,::-1],  lw=lw, color=color2, linestyle=ls1, marker=mark2);
ax.loglog(hSimplePoly[::-1],L2Error_SimplePolyQC[2,::-1],  lw=lw, color='darkblue', linestyle=ls1, marker='x');



#add slope triangle
#Quartic
slope_marker((hSimplePoly[-2],L2Error_SimplePolyQC[2,-2]), (4,1), 0.15, 0.15, ax,  ec='black', linestyle='solid', lw='1.5', fill=False)
#Cubic
slope_marker((hSimplePoly[-2],L2Error_SimplePolyQC[1,-2]), (3,1), 0.15, 0.15, ax,  ec='black', linestyle='solid', lw='1.5', fill=False)
#Quadratic
slope_marker((hSimplePoly[-3],L2Error_SimplePolyQC[0,-3]), (2,1), 0.15, 0.15, ax, ec='black', linestyle='solid', lw='1.5', fill=False)

#add H1 errors
ax.loglog(hSimplePoly[::-1],H1Error_SimplePolyQC[0,::-1],  lw=lw, color=color1, linestyle=ls2, marker=mark1);
ax.loglog(hSimplePoly[::-1],H1Error_SimplePolyQC[1,::-1],  lw=lw, color=color2, linestyle=ls2, marker=mark2);
ax.loglog(hSimplePoly[::-1],H1Error_SimplePolyQC[2,::-1],  lw=lw, color='darkblue', linestyle=ls2, marker='x');

#add slope triangle for H1 error
#linear
slope_marker((hSimplePoly[-3],H1Error_SimplePolyQC[0,-3]), (1,1), 0.15, 0.15, ax, invert=True, ec='black', linestyle='solid', lw='1.5', fill=False)

ax.legend(("$ L^2\; \mathcal{P}_1$ FE", "$L^2\;\mathcal{P}_2$ FE", "$L^2\;\mathcal{P}_3$ FE",\
          "$ H^1\; \mathcal{P}_1$ FE", "$H^1\;\mathcal{P}_2$ FE", "$H^1\;\mathcal{P}_3$ FE",), loc='lower right', framealpha=0.7)
ax.set_xlabel("Mesh width");
ax.set_ylabel("errors");
#set tick labels
ax.set_xticks(xTicksSimple)
ax.set_xticklabels(xTickLabSimple, size='14', weight='bold')
ax.set_xlim(min(xTicksSimple), max(xTicksSimple));

ax.grid();
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
plt.title("Poisson with polynomial solution")
fig.tight_layout();
plt.show()

Do the same for the trig equation

In [55]:
fig, ax = plt.subplots()

ax.loglog(hTrig[::-1],L2Error_TrigQC[0,::-1],  lw=lw, color=color1, linestyle=ls1, marker=mark1);
ax.loglog(hTrig[::-1],L2Error_TrigQC[1,::-1],  lw=lw, color=color2, linestyle=ls1, marker=mark2);
ax.loglog(hTrig[::-1],L2Error_TrigQC[2,::-1],  lw=lw, color='darkblue', linestyle=ls1, marker='x');



#add slope triangle
#Quartic
slope_marker((hTrig[-2],L2Error_TrigQC[2,-2]), (4,1), 0.15, 0.15, ax,  ec='black', linestyle='solid', lw='1.5', fill=False)
#Cubic
slope_marker((hTrig[-2],L2Error_TrigQC[1,-2]), (3,1), 0.15, 0.15, ax,  ec='black', linestyle='solid', lw='1.5', fill=False)
#Quadratic
slope_marker((hTrig[-3],L2Error_TrigQC[0,-3]), (2,1), 0.15, 0.15, ax, ec='black', linestyle='solid', lw='1.5', fill=False)

#add H1 errors
ax.loglog(hTrig[::-1],H1Error_TrigQC[0,::-1],  lw=lw, color=color1, linestyle=ls2, marker=mark1);
ax.loglog(hTrig[::-1],H1Error_TrigQC[1,::-1],  lw=lw, color=color2, linestyle=ls2, marker=mark2);
ax.loglog(hTrig[::-1],H1Error_TrigQC[2,::-1],  lw=lw, color='darkblue', linestyle=ls2, marker='x');

#add slope triangle for H1 error
#linear
slope_marker((hTrig[-3],H1Error_TrigQC[0,-3]), (1,1), 0.15, 0.15, ax, invert=True, ec='black',\
             linestyle='solid', lw='1.5', fill=False)

ax.legend(("$ L^2\; \mathcal{P}_1$ FE", "$L^2\;\mathcal{P}_2$ FE", "$L^2\;\mathcal{P}_3$ FE",\
          "$ H^1\; \mathcal{P}_1$ FE", "$H^1\;\mathcal{P}_2$ FE", "$H^1\;\mathcal{P}_3$ FE",),\
          loc='lower right', framealpha=0.7)

ax.set_xlabel("Mesh width");
ax.set_ylabel("errors");
#set tick labels
ax.set_xticks(xTicksTrig)
ax.set_xticklabels(xTickLabTrig, size='14', weight='bold')
ax.set_xlim(min(xTicksTrig), max(xTicksTrig));

ax.grid();
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
plt.title("Poisson with trigonometric solution")
fig.tight_layout();
plt.show()

And for the transport equation

In [56]:
fig, ax = plt.subplots()

ax.loglog(hTrans[::-1],L2Error_Trans[0,::-1],  lw=lw, color=color1, linestyle=ls1, marker=mark1);
ax.loglog(hTrans[::-1],L2Error_Trans[1,::-1],  lw=lw, color=color2, linestyle=ls1, marker=mark2);
ax.loglog(hTrans[::-1],L2Error_Trans[2,::-1],  lw=lw, color='darkblue', linestyle=ls1, marker='x');



#add slope triangle
#Quartic
slope_marker((hTrans[-2],L2Error_Trans[2,-2]), (4,1), 0.15, 0.15, ax,  ec='black', linestyle='solid', lw='1.5', fill=False)
#Cubic
slope_marker((hTrans[-2],L2Error_Trans[1,-2]), (3,1), 0.15, 0.15, ax,  ec='black', linestyle='solid', lw='1.5', fill=False)
#Quadratic
slope_marker((hTrans[-3],L2Error_Trans[0,-3]), (2,1), 0.15, 0.15, ax, ec='black', linestyle='solid', lw='1.5', fill=False)

#add H1 errors
ax.loglog(hTrans[::-1],H1Error_Trans[0,::-1],  lw=lw, color=color1, linestyle=ls2, marker=mark1);
ax.loglog(hTrans[::-1],H1Error_Trans[1,::-1],  lw=lw, color=color2, linestyle=ls2, marker=mark2);
ax.loglog(hTrans[::-1],H1Error_Trans[2,::-1],  lw=lw, color='darkblue', linestyle=ls2, marker='x');

#add slope triangle for H1 error
#linear
slope_marker((hTrans[-3],H1Error_Trans[0,-3]), (1,1), 0.15, 0.15, ax, invert=True, ec='black',\
             linestyle='solid', lw='1.5', fill=False)

ax.legend(("$ L^2\; \mathcal{P}_1$ FE", "$L^2\;\mathcal{P}_2$ FE", "$L^2\;\mathcal{P}_3$ FE",\
          "$ H^1\; \mathcal{P}_1$ FE", "$H^1\;\mathcal{P}_2$ FE", "$H^1\;\mathcal{P}_3$ FE",),\
          loc='lower right', framealpha=0.7)

ax.set_xlabel("Mesh width");
ax.set_ylabel("errors");
#set tick labels
ax.set_xticks(xTicksTrans)
ax.set_xticklabels(xTickLabTrans, size='14', weight='bold')
ax.set_xlim(min(xTicksTrans), max(xTicksTrans));

ax.grid();
#label size
plt.tick_params(axis='both', which='major', labelsize=lsize)
plt.title("Poisson with transport term")
fig.tight_layout();
plt.show()